In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import re
import json
import sys 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
# import stemmer as hindi_stemmer


In [ ]:
!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 691 kB 5.0 MB/s 
     |████████████████████████████████| 216 kB 50.4 MB/s 
  Created wheel for emoji: filename=emoji-2.1.0-py3-none-any.whl size=212392 sha256=df8aade6058409c2ef54a88bbad2659e2d7e9e05a52e61b41d09dad1dceba827
  Stored in directory: /root/.cache/pip/wheels/77/75/99/51c2a119f4cfd3af7b49cc57e4f737bed7e40b348a85d82804
Successfully built emoji


In [ ]:
import stanza

In [ ]:
stanza.download('ta')

INFO:stanza:Downloading default packages for language: ta (Tamil) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


In [ ]:
nlp = stanza.Pipeline(lang='mr', processors='tokenize,mwt,pos,lemma')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: mr (Marathi):
| Processor | Package |
-----------------------
| tokenize  | ufal    |
| mwt       | ufal    |
| pos       | ufal    |
| lemma     | ufal    |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/HASOC/Train/MOLD_train/MOLD_train.tsv',sep = '\t')

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/hasoc_test/marathi_test_without_labels.csv')

In [ ]:
df_test

,Unnamed: 0,id,tweet,subtask_a,subtask_b,subtask_c
0,0,0,पूर्व लडाखमधील मुखपरी येथे प्रत्यक्ष ताबारेषेज...,NOT,NaN,NaN
1,1,1,कोणत्याही रिलेशनशिप मध्ये सुंदर दिसणं खूप महत...,NOT,NaN,NaN
2,2,2,भारत ऑगस्ट ला स्वतंत्र झाला आणि त्यानंतर तब्...,NOT,NaN,NaN
3,3,3,स्वत ला हवा तसा बाइट किंवा प्रतिक्रिया घेण्यास...,NOT,NaN,NaN
4,4,4,व्या नंबरची अर्थव्यवस्था आहे भारताची जगात पर्...,NOT,NaN,NaN
...,...,...,...,...,...,...
505,505,505,चायला हा मराठीत कधी पासून ट्विट करायला लागला ...,OFF,UIN,NaN
506,506,506,मदत तातडीने द्यायला हवी महिने अधिकारी गोट्या ...,OFF,UIN,NaN
507,507,507,@USER @USER @USER @USER @USER रंगा बिल्ला ने श...,OFF,UIN,NaN
508,508,508,@USER काही लोकं अजूनही म्हणतात की पूर आल्यावर ...,OFF,UIN,NaN


In [ ]:
labels = df_test.subtask_c

In [ ]:
labels

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
505    NaN
506    NaN
507    NaN
508    NaN
509    NaN
Name: subtask_c, Length: 510, dtype: object

In [ ]:
id = df_test.id

In [ ]:
id

0        0
1        1
2        2
3        3
4        4
      ... 
505    505
506    506
507    507
508    508
509    509
Name: id, Length: 510, dtype: int64

In [ ]:
df_submit =  pd.DataFrame(list(zip(id,labels)), columns = ['id','label'])

In [ ]:
df_submit

,id,label
0,0,NaN
1,1,NaN
2,2,NaN
3,3,NaN
4,4,NaN
...,...,...
505,505,UIN
506,506,UIN
507,507,UIN
508,508,UIN


In [ ]:
df_submit.to_csv('task5.csv', index = False)

In [ ]:
def lemmatize_text(str):
  doc= nlp(str)
  List= [word.lemma for sent in doc.sentences for word in sent.words ]
  s= ' '
  s = s.join(List)
  return s

In [ ]:
df

,id,tweet,subtask_a,subtask_b,subtask_c
0,0,आजच्या जनता दरबारात जळगाव जिल्ह्यातील चाळीसगाव...,NOT,NaN,NaN
1,1,कुणी कविता करत असतं तर कुणी कविता जगत असतं कुण...,NOT,NaN,NaN
2,2,आम्हाला इतिहासातील औरंगजेबशी काही घेणे नाही आम...,NOT,NaN,NaN
3,3,गँभीर प्रकरण महाराष्ट्राची अवस्था बिकट आहे भाष...,NOT,NaN,NaN
4,4,कब्झा हा कन्नड चित्रपट लवकरच मराठी मध्ये डब्ब ...,NOT,NaN,NaN
...,...,...,...,...,...
3098,3098,@USER सगळ्यात जास्त वास मारणारी पुच्ची,OFF,UNT,NaN
3099,3099,@USER दोन नंबर पुच्ची पाकळ्या मस्त आहेत चाटायल...,OFF,TIN,OTH
3100,3100,@USER पुच्ची कप्तान साब,OFF,TIN,OTH
3101,3101,@USER नंबरकाळी पुच्ची आणि वर थोडे केसखालून चाट...,OFF,TIN,OTH


In [ ]:
df.subtask_a.value_counts()

0    2034
1    1068
Name: subtask_a, dtype: int64

In [ ]:
df.subtask_b.value_counts()

TIN    741
UNT    327
Name: subtask_b, dtype: int64

In [ ]:
df.subtask_c.value_counts()

IND    503
GRP    157
OTH     80
Name: subtask_c, dtype: int64

In [ ]:
df= df[df['tweet'].isnull()==False]

In [ ]:
df= df[df['subtask_b'] == 'TIN']

In [ ]:
df= df[df['subtask_c'].isnull()==False]

In [ ]:
d_b = {'TIN' : 1 , 'UNT' : 2, 0 : 0}

In [ ]:
d_c = {0 : 0 , 'IND' : 1 , 'GRP' : 2, 'OTH' : 3}

In [ ]:
d_a = {'NOT' : 0 , 'OFF' : 1}

In [ ]:
df['subtask_b'] = df['subtask_b'].fillna(0)

In [ ]:
df['subtask_c'] = df['subtask_c'].fillna(0)

In [ ]:
df.subtask_c = df.subtask_c.map(d_c)

In [ ]:
df.subtask_a = df.subtask_a.map(d_a)

In [ ]:
df.subtask_b = df.subtask_b.map(d_b)

In [ ]:
regex_for_english_hindi_emojis="[^a-zA-Z#\U0001F300-\U0001F5FF'|'\U0001F600-\U0001F64F'|'\U0001F680-\U0001F6FF'|'\u2600-\u26FF\u2700-\u27BF\u0900-\u097F]"
def clean_tweet(tweet):
    tweet = re.sub(r"@[A-Za-z0-9]+",' ', tweet)
    tweet = re.sub(r"https?://[A-Za-z0-9./]+",' ', tweet)
    tweet = re.sub(regex_for_english_hindi_emojis,' ', tweet)
    tweet = re.sub("RT ", " ", tweet)
    tweet = re.sub("\n", " ", tweet)
    tweet = re.sub(r" +", " ", tweet)

    return tweet

In [ ]:
tweets = df.tweet

In [ ]:
cleaned_tweets = [clean_tweet(tweet) for tweet in tweets]

In [ ]:
lemmitized_tweets = [lemmatize_text(tweet) for tweet in cleaned_tweets]

In [ ]:
lemmatize_text(tweets[0])

'आजचा ्य जनण दरबार जळग जिल्हा तील चाळ वेणे रहिव माजी सैनिक सोनणे महाजन येणे असणे २०१६ साली भाजप चा तत्कालीन …'

In [ ]:
lemmatize_text(tweets[0])

'सैनिक तील चाळ … ्य माजी महाजन तत्कालीन जनण येणे भाजप चा २०१६ रहिव सोनणे असणे जळग साली दरबार आजचा जिल्हा वेणे'

In [ ]:
cleaned_tweets[0]

'आजच्या जनता दरबारात जळगाव जिल्ह्यातील चाळीसगावचे रहिवासी माजी सैनिक सोनू महाजन आले होते २०१६ साली भाजपचे तत्कालीन '

In [ ]:
tweets[0]

'आजच्या जनता दरबारात जळगाव जिल्ह्यातील चाळीसगावचे रहिवासी माजी सैनिक सोनू महाजन आले होते २०१६ साली भाजपचे तत्कालीन…'

In [ ]:
labels_a = df.subtask_a

In [ ]:
labels_b = df.subtask_b

In [ ]:
labels_c = df.subtask_c

In [ ]:
labels_a

In [ ]:
labels_b

In [ ]:
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(lemmitized_tweets)
# X = X.todense()

In [ ]:
df_test

,id,tweet,subtask_a,subtask_b,subtask_c
0,0,पूर्व लडाखमधील मुखपरी येथे प्रत्यक्ष ताबारेषेज...,NOT,NaN,NaN
1,1,कोणत्याही रिलेशनशिप मध्ये सुंदर दिसणं खूप महत...,NOT,NaN,NaN
2,2,भारत ऑगस्ट ला स्वतंत्र झाला आणि त्यानंतर तब्...,NOT,NaN,NaN
3,3,स्वत ला हवा तसा बाइट किंवा प्रतिक्रिया घेण्यास...,NOT,NaN,NaN
4,4,व्या नंबरची अर्थव्यवस्था आहे भारताची जगात पर्...,NOT,NaN,NaN
...,...,...,...,...,...
505,505,चायला हा मराठीत कधी पासून ट्विट करायला लागला ...,OFF,UIN,NaN
506,506,मदत तातडीने द्यायला हवी महिने अधिकारी गोट्या ...,OFF,UIN,NaN
507,507,@USER @USER @USER @USER @USER रंगा बिल्ला ने श...,OFF,UIN,NaN
508,508,@USER काही लोकं अजूनही म्हणतात की पूर आल्यावर ...,OFF,UIN,NaN


In [ ]:
df_test = df_test.drop(['Unnamed: 0'], axis=1)

In [ ]:
d_b_final = {1:'TIN' , 2 :'UIN', 0 : np.nan}

In [ ]:
label_b_test = list(df_test['subtask_b_num'])

In [ ]:
label_b_test

In [ ]:
df_test['subtask_b']= df_test['subtask_b_num'].map(d_b_final)

In [ ]:
df_test['subtask_c_num'] = y_pred_c_final

In [ ]:
d_c_final = {0 : np.nan , 1 : 'IND' ,2 : 'GRP', 3 : 'OTH'}

In [ ]:
df_test['subtask_c']= df_test['subtask_c_num'].map(d_c_final)

In [ ]:
labels_a_test = list(df_test['subtask_a_num'])

In [ ]:
labels_a_test

In [ ]:
df_test= df_test[df_test['subtask_a'] == 'OFF']

In [ ]:
df_test.id

0        0
1        1
2        2
3        3
4        4
      ... 
505    505
506    506
507    507
508    508
509    509
Name: id, Length: 510, dtype: int64

In [ ]:
tweets_test = df_test.tweet

In [ ]:
tweets_test

In [ ]:
cleaned_tweets_test = [clean_tweet(tweet) for tweet in tweets_test]

In [ ]:
lemmitized_tweets_test = [lemmatize_text(tweet) for tweet in cleaned_tweets_test]

In [ ]:
X_test = vectorizer.transform(lemmitized_tweets_test)

In [ ]:
df

# ***task a***

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, labels_a, test_size=0.2, random_state=42)


In [ ]:
classifier = KNeighborsClassifier(4)
classifier.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=4)

In [ ]:
y_pred = classifier.predict(X_val)
print("With K-nearest Neighbour:")
print(classification_report(y_val, y_pred))



In [ ]:
y_pred = classifier.predict(X_val)
print("With K-nearest Neighbour:")
print(classification_report(y_val, y_pred))



LogisticRegression 

In [ ]:
reg = LogisticRegression(C = 25,penalty ='l2'  , solver = 'newton-cg' , random_state = 42)

In [ ]:
reg.fit(X, labels_a)

ValueError: ignored

In [ ]:
y_pred = reg.predict(X_val)
print("With K-Nearest Neighbour:")
print(classification_report(y_val, y_pred))

With K-Nearest Neighbour:
              precision    recall  f1-score   support

           0       0.94      0.97      0.96       409
           1       0.94      0.88      0.91       212

    accuracy                           0.94       621
   macro avg       0.94      0.93      0.93       621
weighted avg       0.94      0.94      0.94       621



In [ ]:
# f = open("result/lg_agn.txt",'w')
# f.write(f"LogisticRegression(C = {0},tol = {0},penalty ={0})")
# f.close()
from sklearn.metrics import f1_score
C = [1,5,25,50,100,200]
penalty = ["l2" , 'l1' ,'elasticnet']
solver = ['newton-cg', 'lbfgs', 'sag','liblinear','saga']
# l1_ratio = [0.0,0.3,0.5,0.8,1.0]

In [ ]:
score = 0
for i in C:
  for k in penalty:
    for l in solver :
      print(i,k,l)
      try:
        model = LogisticRegression(C = i,penalty =k  , solver = l , random_state = 42)
        model.fit(X_train,y_train)
      except :
        continue
      pred_m = model.predict(X_val)
      f = f1_score(y_val,pred_m,average = 'macro')
      if(score<f):
          score = f
          # f = open("result/lg_agn.txt",'w')
          # f.write(f"LogisticRegression(C = {i},penalty = {k},solver={l})")
          # f.close()
          # joblib.dump(model,"result/agn_lg.joblib")
          print(i,k,l,score)
          # print(score)
              

SVM

In [ ]:
from sklearn.svm import SVC

In [ ]:
model3 = SVC(C = 10 ,kernel= 'rbf')
model3.fit(X,labels_a)

SVC(C=10)

In [ ]:
import joblib

In [ ]:
joblib.dump(model3,'/content/drive/MyDrive/hasoc_test/svm_mold_subtask_a.joblib')

['/content/drive/MyDrive/hasoc_test/svm_mold_subtask_a.joblib']

In [ ]:
y_pred = model3.predict(X)

ValueError: ignored

In [ ]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
df_test_final = pd.DataFrame(list(zip(df_test.id,tweets_test, y_pred)),columns =['id', 'tweet','subtask_a'])

In [ ]:
d_a = { 0 : 'NOT' , 1 : 'OFF'}

In [ ]:
df_test_final.subtask_a = df_test_final.subtask_a.map(d_a)

In [ ]:
df_test_final

,id,tweet,subtask_a
0,0,पूर्व लडाखमधील मुखपरी येथे प्रत्यक्ष ताबारेषेज...,NOT
1,1,कोणत्याही रिलेशनशिप मध्ये सुंदर दिसणं खूप महत...,NOT
2,2,भारत ऑगस्ट ला स्वतंत्र झाला आणि त्यानंतर तब्...,NOT
3,3,स्वत ला हवा तसा बाइट किंवा प्रतिक्रिया घेण्यास...,NOT
4,4,व्या नंबरची अर्थव्यवस्था आहे भारताची जगात पर्...,NOT
...,...,...,...
505,505,चायला हा मराठीत कधी पासून ट्विट करायला लागला ...,OFF
506,506,मदत तातडीने द्यायला हवी महिने अधिकारी गोट्या ...,OFF
507,507,@USER @USER @USER @USER @USER रंगा बिल्ला ने श...,OFF
508,508,@USER काही लोकं अजूनही म्हणतात की पूर आल्यावर ...,OFF


In [ ]:
df_test.to_csv('/content/drive/MyDrive/hasoc_test/marathi_test_without_labels.csv')

In [ ]:
y_pred = model3.predict(X)
print("With K-Nearest Neighbour:")
print(classification_report(labels_a, y_pred))

With K-Nearest Neighbour:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      2034
           1       0.97      0.98      0.98      1068

    accuracy                           0.98      3102
   macro avg       0.98      0.98      0.98      3102
weighted avg       0.98      0.98      0.98      3102



In [ ]:
mx = 0
for kernel in ['linear', 'poly', 'rbf']:
  for C in [1,5,10,25,50,100,200]:
    # if(kernel=='poly'):
      # if(C != 200):
      for degree in [1,2,3,4]:
          model = SVC(C = C,kernel=kernel,degree=degree)
          model.fit(X_train,y_train)
          pred = model.predict(X_val)
          f = f1_score(y_val,pred,average='macro')
          if(mx<f):
              mx = max(mx,f)
              # s = open('result/gen_svm.txt' , 'w')
              # s.write(f"SVC(C= {C}, degree = {degree} , kernel = {kernel}) score={mx}")
              # s.close()
              # joblib.dump(model,'result/gen_svm.joblib')
              print(kernel,C,degree,mx)
          print(f"C = {C} , kernel = {kernel} , degree = {degree}")
    # else:
          model = SVC(C = C,kernel=kernel)
          model.fit(X_train,y_train)
          pred = model.predict(X_val)
          f = f1_score(y_val,pred,average='macro')
          if(mx<f):
              mx = max(mx,f)
              # s = open('result/gen_svm.txt' , 'w')
              # s.write(f"SVC(C= {C}, degree = {1} , kernal = {kernel}) score={mx}")
              # s.close()
              # joblib.dump(model,'result/gen_svm.joblib')
              print(kernel,C,degree,mx)
          print(f"C = {C} , kernel = {kernel}")

linear 1 1 0.7598328517392412
C = 1 , kernel = linear , degree = 1
C = 1 , kernel = linear
C = 1 , kernel = linear , degree = 2
C = 1 , kernel = linear
C = 1 , kernel = linear , degree = 3
C = 1 , kernel = linear
C = 1 , kernel = linear , degree = 4
C = 1 , kernel = linear
linear 5 1 0.7621502816602634
C = 5 , kernel = linear , degree = 1
C = 5 , kernel = linear
C = 5 , kernel = linear , degree = 2
C = 5 , kernel = linear
C = 5 , kernel = linear , degree = 3
C = 5 , kernel = linear
C = 5 , kernel = linear , degree = 4
C = 5 , kernel = linear
C = 10 , kernel = linear , degree = 1
C = 10 , kernel = linear
C = 10 , kernel = linear , degree = 2
C = 10 , kernel = linear
C = 10 , kernel = linear , degree = 3
C = 10 , kernel = linear
C = 10 , kernel = linear , degree = 4
C = 10 , kernel = linear
C = 25 , kernel = linear , degree = 1
C = 25 , kernel = linear
C = 25 , kernel = linear , degree = 2
C = 25 , kernel = linear
C = 25 , kernel = linear , degree = 3
C = 25 , kernel = linear
C = 25 , ke

MNB

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
model = MultinomialNB(alpha=0.06891)
model.fit(X_train,y_train)

MultinomialNB(alpha=0.06891)

In [ ]:
y_pred = model.predict(X_val)
# print("With K-Nearest Neighbour:")
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.92      0.86       409
           1       0.79      0.58      0.67       212

    accuracy                           0.80       621
   macro avg       0.80      0.75      0.76       621
weighted avg       0.80      0.80      0.79       621



In [ ]:

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
score =0
for i in np.arange(1e-05,1e-01,1e-04):
  print(i)
  model = MultinomialNB(alpha=i)
  model.fit(X_train,y_train)
  pred = model.predict(X_val)
  f = f1_score(y_val,pred,average='macro')
  if(score<f):
    score = f
    # f = open("result/gen_mnb.txt",'w')
    # f.write(f"MultinomialNB(alpha = {i})")
    # f.close()
    # joblib.dump(model,'result/gen_mnb.joblib')
    print(i,score)

1e-05
1e-05 0.7436090855753918
0.00011
0.00021
0.00021 0.7451372997711669
0.00031000000000000005
0.00041000000000000005
0.00041000000000000005 0.7489988558352403
0.00051
0.0006100000000000001
0.00071
0.0008100000000000001
0.0009100000000000001
0.00101
0.00111
0.0012100000000000001
0.0013100000000000002
0.00141
0.00151
0.00161
0.0017100000000000001
0.0018100000000000002
0.00191
0.00201
0.0021100000000000003
0.00221
0.00231
0.0024100000000000002
0.00251
0.0026100000000000003
0.00271
0.00281
0.00281 0.7513200905204929
0.0029100000000000003
0.00301
0.0031100000000000004
0.00321
0.00331
0.0034100000000000003
0.00351
0.0036100000000000004
0.00371
0.00381
0.00391
0.00401
0.00411
0.00421
0.00431
0.00441
0.00451
0.0046099999999999995
0.00471
0.00481
0.0049099999999999994
0.00501
0.00511
0.00521
0.00531
0.00541
0.00551
0.0056099999999999995
0.00571
0.00581
0.0059099999999999995
0.00601
0.00611
0.00621
0.00631
0.00641
0.00651
0.0066099999999999996
0.00671
0.00681
0.00691
0.00691 0.752860411899313

randomforest

In [ ]:
from sklearn.ensemble import RandomForestClassifier 

In [ ]:
from sklearn.metrics import f1_score
max_features = ['auto', 'sqrt', 'log2']
max_depth = [4,5,6,7,8]
criterion =['gini', 'entropy'] 

In [ ]:
score=0
for i in max_features:
  for j in max_depth:
    for k in criterion:
      print(i,j,k)
      model=RandomForestClassifier(max_features=i,max_depth=j,criterion=k)
      model.fit(X_train,y_train)
      pred=model.predict(X_val)
      f=f1_score(y_val,pred,average='macro')
      if(score<f):
        score=f
        # f=open("result/gen_rf.txt",'w')
        # f.write(f"RandomForestClassifier(C={i},tol={j},penalty={k}) score={score}")
        # f.close()
        # joblib.dump(model,'result/gen_rf.joblib')
        print(i,j,k,score)

auto 4 gini
auto 4 gini 0.3970873786407767
auto 4 entropy
auto 5 gini
auto 5 entropy
auto 6 gini
auto 6 entropy
auto 7 gini
auto 7 entropy
auto 7 entropy 0.4072057165715117
auto 8 gini
auto 8 entropy
auto 8 entropy 0.4122008106700482
sqrt 4 gini
sqrt 4 entropy
sqrt 5 gini
sqrt 5 entropy
sqrt 6 gini
sqrt 6 entropy
sqrt 7 gini
sqrt 7 entropy
sqrt 8 gini
sqrt 8 entropy
sqrt 8 entropy 0.42206586489828035
log2 4 gini
log2 4 entropy
log2 5 gini
log2 5 entropy
log2 6 gini
log2 6 entropy
log2 7 gini
log2 7 entropy
log2 8 gini
log2 8 entropy


In [ ]:
model=RandomForestClassifier(class_weight= classweight)
model.fit(X_train,y_train)

RandomForestClassifier(class_weight={0: 0.7625368731563422,
                                     1: 1.452247191011236})

In [ ]:
y_pred = model.predict(X_val)
# print("With K-Nearest Neighbour:")
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.89      0.86       409
           1       0.76      0.65      0.70       212

    accuracy                           0.81       621
   macro avg       0.80      0.77      0.78       621
weighted avg       0.81      0.81      0.81       621



# **task b**

In [ ]:
#applying class weight
from sklearn.utils import compute_class_weight
classweight = dict(zip([1,2],compute_class_weight(class_weight = 'balanced',y=df['subtask_b'].values,classes=[1,2])))
classweight

{1: 0.720945945945946, 2: 1.6314984709480123}

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, labels_b, test_size=0.2, random_state=42)

In [ ]:
classifier = KNeighborsClassifier(4)
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_val)
print("With K-nearest Neighbour:")
print(classification_report(y_val, y_pred))



LogisticRegression 

In [ ]:
reg = LogisticRegression(C = 50,penalty ='l1'  , solver = 'saga' , random_state = 42,class_weight = classweight)

In [ ]:
reg.fit(X,labels_b)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


LogisticRegression(C=50,
                   class_weight={1: 0.720945945945946, 2: 1.6314984709480123},
                   penalty='l1', random_state=42, solver='saga')

In [ ]:
y_pred = reg.predict(X)
print("With K-Nearest Neighbour:")
print(classification_report(labels_b, y_pred))

With K-Nearest Neighbour:
              precision    recall  f1-score   support

           1       0.98      0.95      0.96       740
           2       0.89      0.96      0.92       327

    accuracy                           0.95      1067
   macro avg       0.93      0.95      0.94      1067
weighted avg       0.95      0.95      0.95      1067



In [ ]:
y_pred_b = reg.predict(X_test)

In [ ]:
 y_pred_b_final = y_pred_b

In [ ]:
range(len(labels_a_test)-1)

range(0, 509)

In [ ]:
for i in range(len(labels_a_test)):
  if(labels_a_test[i] == 0):
    y_pred_b_final[i] = 0
  # print(i) 

In [ ]:
len(y_pred_b_final)

510

In [ ]:
len(y_pred_b)

510

NameError: ignored

In [ ]:
# f = open("result/lg_agn.txt",'w')
# f.write(f"LogisticRegression(C = {0},tol = {0},penalty ={0})")
# f.close()
from sklearn.metrics import f1_score
C = [1,5,25,50,100,200]
penalty = ["l2" , 'l1' ,'elasticnet']
solver = ['newton-cg', 'lbfgs', 'sag','liblinear','saga']
# l1_ratio = [0.0,0.3,0.5,0.8,1.0]

In [ ]:
score = 0
for i in C:
  for k in penalty:
    for l in solver :
      print(i,k,l)
      try:
        model = LogisticRegression(C = i,penalty =k  , solver = l , random_state = 42, class_weight = classweight)
        model.fit(X_train,y_train)
      except :
        continue
      pred_m = model.predict(X_val)
      f = f1_score(y_val,pred_m,average = 'macro')
      if(score<f):
          score = f
          # f = open("result/lg_agn.txt",'w')
          # f.write(f"LogisticRegression(C = {i},penalty = {k},solver={l})")
          # f.close()
          # joblib.dump(model,"result/agn_lg.joblib")
          print(i,k,l,score)
          # print(score)
              

svm

In [ ]:
from sklearn.svm import SVC

In [ ]:
model3 = SVC(C = 100 ,kernel= 'linear')
model3.fit(X_train,y_train)

SVC(C=100, kernel='linear')

In [ ]:
y_pred = model3.predict(X_val)
print("With K-Nearest Neighbour:")
print(classification_report(y_val, y_pred))

With K-Nearest Neighbour:
              precision    recall  f1-score   support

           1       0.74      0.77      0.75       141
           2       0.51      0.48      0.50        73

    accuracy                           0.67       214
   macro avg       0.63      0.62      0.62       214
weighted avg       0.66      0.67      0.67       214



In [ ]:
mx = 0
for kernel in ['linear', 'poly', 'rbf']:
  for C in [1,5,10,25,50,100,200]:
    # if(kernel=='poly'):
      # if(C != 200):
      for degree in [1,2,3,4]:
          model = SVC(C = C,kernel=kernel,degree=degree,class_weight = classweight)
          model.fit(X_train,y_train)
          pred = model.predict(X_val)
          f = f1_score(y_val,pred,average='macro')
          if(mx<f):
              mx = max(mx,f)
              # s = open('result/gen_svm.txt' , 'w')
              # s.write(f"SVC(C= {C}, degree = {degree} , kernel = {kernel}) score={mx}")
              # s.close()
              # joblib.dump(model,'result/gen_svm.joblib')
              print(kernel,C,degree,mx)
          print(f"C = {C} , kernel = {kernel} , degree = {degree}")
    # else:
          model = SVC(C = C,kernel=kernel,degree = degree, class_weight = classweight)
          model.fit(X_train,y_train)
          pred = model.predict(X_val)
          f = f1_score(y_val,pred,average='macro')
          if(mx<f):
              mx = max(mx,f)
              # s = open('result/gen_svm.txt' , 'w')
              # s.write(f"SVC(C= {C}, degree = {1} , kernal = {kernel}) score={mx}")
              # s.close()
              # joblib.dump(model,'result/gen_svm.joblib')
              print(kernel,C,degree,mx)
          print(f"C = {C} , kernel = {kernel}")

linear 1 1 0.5958113180481508
C = 1 , kernel = linear , degree = 1
C = 1 , kernel = linear
C = 1 , kernel = linear , degree = 2
C = 1 , kernel = linear
C = 1 , kernel = linear , degree = 3
C = 1 , kernel = linear
C = 1 , kernel = linear , degree = 4
C = 1 , kernel = linear
C = 5 , kernel = linear , degree = 1
C = 5 , kernel = linear
C = 5 , kernel = linear , degree = 2
C = 5 , kernel = linear
C = 5 , kernel = linear , degree = 3
C = 5 , kernel = linear
C = 5 , kernel = linear , degree = 4
C = 5 , kernel = linear
C = 10 , kernel = linear , degree = 1
C = 10 , kernel = linear
C = 10 , kernel = linear , degree = 2
C = 10 , kernel = linear
C = 10 , kernel = linear , degree = 3
C = 10 , kernel = linear
C = 10 , kernel = linear , degree = 4
C = 10 , kernel = linear
C = 25 , kernel = linear , degree = 1
C = 25 , kernel = linear
C = 25 , kernel = linear , degree = 2
C = 25 , kernel = linear
C = 25 , kernel = linear , degree = 3
C = 25 , kernel = linear
C = 25 , kernel = linear , degree = 4
C =

In [ ]:
score

0.5710756267845539

mnb

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
model = MultinomialNB(alpha=1e-05)
model.fit(X_train,y_train)

TypeError: ignored

In [ ]:
y_pred = model.predict(X_val)
# print("With K-Nearest Neighbour:")
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           1       0.69      0.84      0.75       141
           2       0.45      0.26      0.33        73

    accuracy                           0.64       214
   macro avg       0.57      0.55      0.54       214
weighted avg       0.61      0.64      0.61       214



In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
score =0
for i in np.arange(1e-05,1e-01,1e-04):
  print(i)
  model = MultinomialNB(alpha=i)
  model.fit(X_train,y_train)
  pred = model.predict(X_val)
  f = f1_score(y_val,pred,average='macro')
  if(score<f):
    score = f
    # f = open("result/gen_mnb.txt",'w')
    # f.write(f"MultinomialNB(alpha = {i})")
    # f.close()
    # joblib.dump(model,'result/gen_mnb.joblib')
    print(i,score)

1e-05
1e-05 0.5422141964161689
0.00011
0.00021
0.00031000000000000005
0.00041000000000000005
0.00051
0.0006100000000000001
0.00071
0.0008100000000000001
0.0009100000000000001
0.00101
0.00111
0.0012100000000000001
0.0013100000000000002
0.00141
0.00151
0.00161
0.0017100000000000001
0.0018100000000000002
0.00191
0.00201
0.0021100000000000003
0.00221
0.00231
0.0024100000000000002
0.00251
0.0026100000000000003
0.00271
0.00281
0.0029100000000000003
0.00301
0.0031100000000000004
0.00321
0.00331
0.0034100000000000003
0.00351
0.0036100000000000004
0.00371
0.00381
0.00391
0.00401
0.00411
0.00421
0.00431
0.00441
0.00451
0.0046099999999999995
0.00471
0.00481
0.0049099999999999994
0.00501
0.00511
0.00521
0.00531
0.00541
0.00551
0.0056099999999999995
0.00571
0.00581
0.0059099999999999995
0.00601
0.00611
0.00621
0.00631
0.00641
0.00651
0.0066099999999999996
0.00671
0.00681
0.00691
0.00701
0.00711
0.00721
0.00731
0.00741
0.00751
0.00761
0.00771
0.00781
0.00791
0.00801
0.00811
0.00821
0.00831
0.00841
0

randomforest

In [ ]:
from sklearn.ensemble import RandomForestClassifier 

In [ ]:
model=RandomForestClassifier()
model.fit(X_train,y_train)

RandomForestClassifier()

In [ ]:
y_pred = model.predict(X_val)
# print("With K-Nearest Neighbour:")
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           1       0.67      0.91      0.77       141
           2       0.43      0.12      0.19        73

    accuracy                           0.64       214
   macro avg       0.55      0.52      0.48       214
weighted avg       0.59      0.64      0.57       214



In [ ]:
from sklearn.metrics import f1_score
max_features = ['auto', 'sqrt', 'log2']
max_depth = [4,5,6,7,8]
criterion =['gini', 'entropy'] 

In [ ]:
score=0
for i in max_features:
  for j in max_depth:
    for k in criterion:
      print(i,j,k)
      model=RandomForestClassifier(max_features=i,max_depth=j,criterion=k)
      model.fit(X_train,y_train)
      pred=model.predict(X_val)
      f=f1_score(y_val,pred,average='macro')
      if(score<f):
        score=f
        # f=open("result/gen_rf.txt",'w')
        # f.write(f"RandomForestClassifier(C={i},tol={j},penalty={k}) score={score}")
        # f.close()
        # joblib.dump(model,'result/gen_rf.joblib')
        print(i,j,k,score)

# **task_c**

In [ ]:
#applying class weight
from sklearn.utils import compute_class_weight
classweight = dict(zip([1,2,3],compute_class_weight(class_weight = 'balanced',y=df['subtask_c'].values,classes=[1,2,3])))
classweight

{1: 0.4907038512616202, 2: 1.5690021231422506, 3: 3.0791666666666666}

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, labels_c, test_size=0.2, random_state=42)

LogisticRegression 

In [ ]:
reg = LogisticRegression(C = 1,penalty ='l2'  , solver = 'liblinear' , random_state = 42, class_weight= classweight)

In [ ]:
reg.fit(X,labels_c)

LogisticRegression(C=1,
                   class_weight={1: 0.4907038512616202, 2: 1.5690021231422506,
                                 3: 3.0791666666666666},
                   random_state=42, solver='liblinear')

In [ ]:
y_pred_c = reg.predict(X_test)

In [ ]:
y_pred_c

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 3, 1, 1,
       1, 3, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 2, 1, 1, 3, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2,
       1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 3, 2, 1, 2, 2, 2,
       1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2,

In [ ]:
y_pred_c_final = y_pred_c

In [ ]:
y_pred_c_final

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 2, 2, 2, 2, 2, 2, 1, 0, 2,
       1, 2, 2, 0, 2, 1, 2, 0, 2, 1, 0, 2, 2, 2, 0, 2, 3, 2, 1, 2, 2, 2,
       1, 0, 1, 1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2,

In [ ]:
for i in range(len(label_b_test)) :
  if(label_b_test[i] != 1) :
    y_pred_c_final[i] = 0


In [ ]:
y_pred = reg.predict(X_val)
print("With K-Nearest Neighbour:")
print(classification_report(y_val, y_pred))

With K-Nearest Neighbour:
              precision    recall  f1-score   support

           1       0.71      0.93      0.81        96
           2       0.82      0.23      0.36        39
           3       0.42      0.38      0.40        13

    accuracy                           0.70       148
   macro avg       0.65      0.51      0.52       148
weighted avg       0.71      0.70      0.65       148



In [ ]:
# f = open("result/lg_agn.txt",'w')
# f.write(f"LogisticRegression(C = {0},tol = {0},penalty ={0})")
# f.close()
from sklearn.metrics import f1_score
C = [1,5,25,50,100,200]
penalty = ["l2" , 'l1' ,'elasticnet']
solver = ['newton-cg', 'lbfgs', 'sag','liblinear','saga']
# l1_ratio = [0.0,0.3,0.5,0.8,1.0]

In [ ]:
score = 0
for i in C:
  for k in penalty:
    for l in solver :
      print(i,k,l)
      try:
        model = LogisticRegression(C = i,penalty =k  , solver = l , random_state = 42, class_weight = classweight)
        model.fit(X_train,y_train)
      except :
        continue
      pred_m = model.predict(X_val)
      f = f1_score(y_val,pred_m,average = 'macro')
      if(score<f):
          score = f
          # f = open("result/lg_agn.txt",'w')
          # f.write(f"LogisticRegression(C = {i},penalty = {k},solver={l})")
          # f.close()
          # joblib.dump(model,"result/agn_lg.joblib")
          print(i,k,l,score)
          # print(score)
              

1 l2 newton-cg
1 l2 newton-cg 0.48399986833876435
1 l2 lbfgs
1 l2 sag
1 l2 liblinear
1 l2 liblinear 0.5218099547511312
1 l2 saga
1 l1 newton-cg
1 l1 lbfgs
1 l1 sag
1 l1 liblinear
1 l1 saga


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


1 elasticnet newton-cg
1 elasticnet lbfgs
1 elasticnet sag
1 elasticnet liblinear
1 elasticnet saga
5 l2 newton-cg
5 l2 lbfgs
5 l2 sag
5 l2 liblinear
5 l2 saga


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


5 l1 newton-cg
5 l1 lbfgs
5 l1 sag
5 l1 liblinear
5 l1 saga


...

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


100 elasticnet newton-cg
100 elasticnet lbfgs
100 elasticnet sag
100 elasticnet liblinear
100 elasticnet saga
200 l2 newton-cg
200 l2 lbfgs


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


200 l2 sag
200 l2 liblinear
200 l2 saga
200 l1 newton-cg
200 l1 lbfgs
200 l1 sag
200 l1 liblinear
200 l1 saga
200 elasticnet newton-cg
200 elasticnet lbfgs
200 elasticnet sag
200 elasticnet liblinear
200 elasticnet saga


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


SVM

In [ ]:
from sklearn.svm import SVC

In [ ]:
model3 = SVC(C = 5 ,kernel= 'rbf')
model3.fit(X_train,y_train)


SVC(C=5)

In [ ]:

y_pred = model3.predict(X_val)
print("With K-Nearest Neighbour:")
print(classification_report(y_val, y_pred))

With K-Nearest Neighbour:
              precision    recall  f1-score   support

           1       0.70      0.94      0.80        96
           2       0.77      0.26      0.38        39
           3       0.50      0.23      0.32        13

    accuracy                           0.70       148
   macro avg       0.66      0.47      0.50       148
weighted avg       0.70      0.70      0.65       148



In [ ]:
mx = 0
for kernel in ['linear', 'poly', 'rbf']:
  for C in [1,5,10,25,50,100,200]:
    # if(kernel=='poly'):
      # if(C != 200):
      for degree in [1,2,3,4]:
          model = SVC(C = C,kernel=kernel,degree=degree)
          model.fit(X_train,y_train)
          pred = model.predict(X_val)
          f = f1_score(y_val,pred,average='macro')
          if(mx<f):
              mx = max(mx,f)
              # s = open('result/gen_svm.txt' , 'w')
              # s.write(f"SVC(C= {C}, degree = {degree} , kernel = {kernel}) score={mx}")
              # s.close()
              # joblib.dump(model,'result/gen_svm.joblib')
              print(kernel,C,degree,mx)
          print(f"C = {C} , kernel = {kernel} , degree = {degree}")
    # else:
          model = SVC(C = C,kernel=kernel,degree= degree)
          model.fit(X_train,y_train)
          pred = model.predict(X_val)
          f = f1_score(y_val,pred,average='macro')
          if(mx<f):
              mx = max(mx,f)
              # s = open('result/gen_svm.txt' , 'w')
              # s.write(f"SVC(C= {C}, degree = {1} , kernal = {kernel}) score={mx}")
              # s.close()
              # joblib.dump(model,'result/gen_svm.joblib')
              print(kernel,C,degree,mx)
          print(f"C = {C} , kernel = {kernel}")

linear 1 1 0.36595744680851067
C = 1 , kernel = linear , degree = 1
C = 1 , kernel = linear
C = 1 , kernel = linear , degree = 2
C = 1 , kernel = linear
C = 1 , kernel = linear , degree = 3
C = 1 , kernel = linear
C = 1 , kernel = linear , degree = 4
C = 1 , kernel = linear
linear 5 1 0.45588235294117646
C = 5 , kernel = linear , degree = 1
C = 5 , kernel = linear
C = 5 , kernel = linear , degree = 2
C = 5 , kernel = linear
C = 5 , kernel = linear , degree = 3
C = 5 , kernel = linear
C = 5 , kernel = linear , degree = 4
C = 5 , kernel = linear
C = 10 , kernel = linear , degree = 1
C = 10 , kernel = linear
C = 10 , kernel = linear , degree = 2
C = 10 , kernel = linear
C = 10 , kernel = linear , degree = 3
C = 10 , kernel = linear
C = 10 , kernel = linear , degree = 4
C = 10 , kernel = linear
C = 25 , kernel = linear , degree = 1
C = 25 , kernel = linear
C = 25 , kernel = linear , degree = 2
C = 25 , kernel = linear
C = 25 , kernel = linear , degree = 3
C = 25 , kernel = linear
C = 25 , 

MNB

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:

model = MultinomialNB(alpha=0.03681)
model.fit(X_train,y_train)

MultinomialNB(alpha=0.03681)

In [ ]:
y_pred = model.predict(X_val)
# print("With K-Nearest Neighbour:")
print(classification_report(y_val, y_pred))


              precision    recall  f1-score   support

           1       0.69      0.76      0.72        96
           2       0.52      0.36      0.42        39
           3       0.27      0.31      0.29        13

    accuracy                           0.61       148
   macro avg       0.49      0.48      0.48       148
weighted avg       0.61      0.61      0.61       148



In [ ]:

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
score =0
for i in np.arange(1e-05,1e-01,1e-04):
  print(i)
  model = MultinomialNB(alpha=i)
  model.fit(X_train,y_train)
  pred = model.predict(X_val)
  f = f1_score(y_val,pred,average='macro')
  if(score<f):
    score = f
    # f = open("result/gen_mnb.txt",'w')
    # f.write(f"MultinomialNB(alpha = {i})")
    # f.close()
    # joblib.dump(model,'result/gen_mnb.joblib')
    print(i,score)

1e-05
1e-05 0.4525023783901807
0.00011
0.00021
0.00031000000000000005
0.00041000000000000005
0.00051
0.0006100000000000001
0.00071
0.0008100000000000001
0.0009100000000000001
0.00101
0.00111
0.0012100000000000001
0.0013100000000000002
0.00141
0.00151
0.00161
0.0017100000000000001
0.0018100000000000002
0.00191
0.00201
0.0021100000000000003
0.00221
0.00231
0.0024100000000000002
0.00251
0.0026100000000000003
0.00271
0.00281
0.0029100000000000003
0.0029100000000000003 0.4686416769412392
0.00301
0.0031100000000000004
0.00321
0.00331
0.0034100000000000003
0.00351
0.0036100000000000004
0.0036100000000000004 0.4703030303030302
0.00371
0.00381
0.00391
0.00401
0.00411
0.00421
0.00431
0.00441
0.00451
0.0046099999999999995
0.00471
0.00481
0.0049099999999999994
0.00501
0.00511
0.00521
0.00531
0.00541
0.00551
0.0056099999999999995
0.00571
0.00581
0.0059099999999999995
0.00601
0.00611
0.00621
0.00631
0.00641
0.00651
0.0066099999999999996
0.00671
0.00681
0.00691
0.00701
0.00711
0.00721
0.00731
0.00741

In [ ]:
score

0.4775763290614776

randomforest

In [ ]:
from sklearn.ensemble import RandomForestClassifier 

In [ ]:
model=RandomForestClassifier(class_weight= classweight)
model.fit(X_train,y_train)

RandomForestClassifier(class_weight={1: 0.4907038512616202,
                                     2: 1.5690021231422506,
                                     3: 3.0791666666666666})

In [ ]:
y_pred = model.predict(X_val)
# print("With K-Nearest Neighbour:")
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           1       0.69      0.94      0.79        96
           2       1.00      0.21      0.34        39
           3       0.22      0.15      0.18        13

    accuracy                           0.68       148
   macro avg       0.64      0.43      0.44       148
weighted avg       0.73      0.68      0.62       148



In [ ]:

from sklearn.metrics import f1_score
max_features = ['auto', 'sqrt', 'log2']
max_depth = [4,5,6,7,8]
criterion =['gini', 'entropy'] 


In [ ]:
score=0
for i in max_features:
  for j in max_depth:
    for k in criterion:
      print(i,j,k)
      model=RandomForestClassifier(max_features=i,max_depth=j,criterion=k)
      model.fit(X_train,y_train)
      pred=model.predict(X_val)
      f=f1_score(y_val,pred,average='macro')
      if(score<f):
        score=f
        # f=open("result/gen_rf.txt",'w')
        # f.write(f"RandomForestClassifier(C={i},tol={j},penalty={k}) score={score}")
        # f.close()
        # joblib.dump(model,'result/gen_rf.joblib')
        print(i,j,k,score)

auto 4 gini
auto 4 gini 0.26229508196721313
auto 4 entropy
auto 5 gini
auto 5 entropy
auto 6 gini
auto 6 gini 0.2800411522633745
auto 6 entropy
auto 7 gini
auto 7 entropy
auto 8 gini
auto 8 entropy
sqrt 4 gini
sqrt 4 entropy
sqrt 5 gini
sqrt 5 entropy
sqrt 6 gini
sqrt 6 entropy
sqrt 7 gini
sqrt 7 entropy
sqrt 8 gini
sqrt 8 gini 0.28112947658402204
sqrt 8 entropy
log2 4 gini
log2 4 entropy
log2 5 gini
log2 5 entropy
log2 6 gini
log2 6 entropy
log2 7 gini
log2 7 entropy
log2 8 gini
log2 8 entropy
